<a href="https://colab.research.google.com/github/samarthk/Learning_pyspark/blob/master/Spark_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Install Java, Spark, and Findspark
This installs Apache Spark 2.3.1, Java 8, and [Findspark](https://github.com/minrk/findspark), a library that makes it easy for Python to find Spark.

In [ ]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget https://archive.apache.org/dist/spark/spark-2.4.1/spark-2.4.1-bin-hadoop2.7.tgz
!tar xf spark-2.4.1-bin-hadoop2.7.tgz
!pip install -q findspark


## Set Environment Variables
Set the locations where Spark and Java are installed.

In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-2.4.1-bin-hadoop2.7"

## Start a SparkSession
This will start a local Spark session.

In [ ]:
# Run a local spark session to test our installation:
import findspark
findspark.init()
from pyspark.sql import SparkSession
spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
import findspark
findspark.init()

from pyspark import SparkContext
from pyspark import SparkConf
from pyspark.sql import SQLContext

conf = SparkConf().setMaster("local[*]").setAppName('pyspark')
sc = SparkContext(conf=conf)
sqc = SQLContext(sc)

## Use Spark!
That's all there is to it - you're ready to use Spark!

In [ ]:
df = spark.createDataFrame([{"hello": "world"} for x in range(1000)])
df.show(3)

+-----+
|hello|
+-----+
|world|
|world|
|world|
+-----+
only showing top 3 rows



/content/spark-2.4.1-bin-hadoop2.7/python/pyspark/sql/session.py:346: UserWarning: inferring schema from dict is deprecated,please use pyspark.sql.Row instead
  warnings.warn("inferring schema from dict is deprecated,"


# Programming Ex


Download File

In [ ]:
!wget https://data.sfgov.org/api/views/wr8u-xric/rows.csv

In [ ]:
!ls -ltr /content/sample_data/

!head -5000 /content/drive/My\ Drive/Data_Files/Fire_Incidents_FULL.csv > Fire_Incidents.csv


In [ ]:
from pyspark.sql.types import *

## San Francisco Fire Incidents (OpenData Meetup)

In [ ]:
fireServiceCallsDF = spark.read.csv('/content/sample_data/Fire_Department_Calls_for_Service.csv', header=True, inferSchema=True)
fireServiceCallsDF.printSchema()

root
 |-- Call Number: integer (nullable = true)
 |-- Unit ID: string (nullable = true)
 |-- Incident Number: integer (nullable = true)
 |-- Call Type: string (nullable = true)
 |-- Call Date: string (nullable = true)
 |-- Watch Date: string (nullable = true)
 |-- Received DtTm: string (nullable = true)
 |-- Entry DtTm: string (nullable = true)
 |-- Dispatch DtTm: string (nullable = true)
 |-- Response DtTm: string (nullable = true)
 |-- On Scene DtTm: string (nullable = true)
 |-- Transport DtTm: string (nullable = true)
 |-- Hospital DtTm: string (nullable = true)
 |-- Call Final Disposition: string (nullable = true)
 |-- Available DtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode of Incident: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- Station Area: integer (nullable = true)
 |-- Box: integer (nullable = true)
 |-- Original Priority: string (nullable = true)
 |-- Priority: string (nullab

In [ ]:
# Note that we are removing all space characters from the col names to prevent errors when writing to Parquet later

fireSchema = StructType([StructField('CallNumber', IntegerType(), True),
                     StructField('UnitID', StringType(), True),
                     StructField('IncidentNumber', IntegerType(), True),
                     StructField('CallType', StringType(), True),                  
                     StructField('CallDate', StringType(), True),       
                     StructField('WatchDate', StringType(), True),       
                     StructField('ReceivedDtTm', StringType(), True),       
                     StructField('EntryDtTm', StringType(), True),       
                     StructField('DispatchDtTm', StringType(), True),       
                     StructField('ResponseDtTm', StringType(), True),       
                     StructField('OnSceneDtTm', StringType(), True),       
                     StructField('TransportDtTm', StringType(), True),                  
                     StructField('HospitalDtTm', StringType(), True),       
                     StructField('CallFinalDisposition', StringType(), True),       
                     StructField('AvailableDtTm', StringType(), True),       
                     StructField('Address', StringType(), True),       
                     StructField('City', StringType(), True),       
                     StructField('ZipcodeofIncident', IntegerType(), True),       
                     StructField('Battalion', StringType(), True),                 
                     StructField('StationArea', StringType(), True),       
                     StructField('Box', StringType(), True),       
                     StructField('OriginalPriority', StringType(), True),       
                     StructField('Priority', StringType(), True),       
                     StructField('FinalPriority', IntegerType(), True),       
                     StructField('ALSUnit', BooleanType(), True),       
                     StructField('CallTypeGroup', StringType(), True),
                     StructField('NumberofAlarms', IntegerType(), True),
                     StructField('UnitType', StringType(), True),
                     StructField('Unitsequenceincalldispatch', IntegerType(), True),
                     StructField('FirePreventionDistrict', StringType(), True),
                     StructField('SupervisorDistrict', StringType(), True),
                     StructField('NeighborhoodDistrict', StringType(), True),
                     StructField('Location', StringType(), True),
                     StructField('RowID', StringType(), True)])

In [ ]:
fireServiceCallsDF_sch = spark.read.csv('/content/sample_data/Fire_Department_Calls_for_Service.csv', header=True, schema=fireSchema)
#fireServiceCallsDF_sch.printSchema()
fireServiceCallsDF_sch.show(5)

+----------+------+--------------+-----------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+---------+-----------+----+----------------+--------+-------------+-------+--------------------+--------------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+--------------+
|CallNumber|UnitID|IncidentNumber|         CallType|  CallDate| WatchDate|        ReceivedDtTm|           EntryDtTm|        DispatchDtTm|        ResponseDtTm|         OnSceneDtTm|       TransportDtTm|        HospitalDtTm|CallFinalDisposition|       AvailableDtTm|             Address|         City|ZipcodeofIncident|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|       CallTypeGroup|NumberofAlarms|UnitType|U

In [ ]:
fireServiceCallsDF.count()

4999

In [ ]:
fireServiceCallsDF_sch.select('CallType').distinct().show(1000,False)

+--------------------------------------------+
|CallType                                    |
+--------------------------------------------+
|Elevator / Escalator Rescue                 |
|Alarms                                      |
|Odor (Strange / Unknown)                    |
|Citizen Assist / Service Call               |
|HazMat                                      |
|Explosion                                   |
|Vehicle Fire                                |
|Extrication / Entrapped (Machinery, Vehicle)|
|Other                                       |
|Outside Fire                                |
|Traffic Collision                           |
|Gas Leak (Natural and LP Gases)             |
|Water Rescue                                |
|Electrical Hazard                           |
|High Angle Rescue                           |
|Structure Fire                              |
|Industrial Accidents                        |
|Medical Incident                            |
|Fuel Spill  

In [ ]:
fireServiceCallsDF_sch.createOrReplaceTempView("firecalls")
spark.sql("SELECT COUNT(*) FROM firecalls").show()

DataFrame[count(1): bigint]

In [ ]:
spark.sql("SELECT * FROM firecalls where \
CallDate = '04/19/2020'and IncidentNumber >20046358 ").show(5)

+----------+------+--------------+----------------+----------+----------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+-------------+-----------------+---------+-----------+----+----------------+--------+-------------+-------+--------------------+--------------+--------+--------------------------+----------------------+------------------+--------------------+--------------------+-------------+
|CallNumber|UnitID|IncidentNumber|        CallType|  CallDate| WatchDate|        ReceivedDtTm|           EntryDtTm|        DispatchDtTm|        ResponseDtTm|         OnSceneDtTm|       TransportDtTm|        HospitalDtTm|CallFinalDisposition|       AvailableDtTm|             Address|         City|ZipcodeofIncident|Battalion|StationArea| Box|OriginalPriority|Priority|FinalPriority|ALSUnit|       CallTypeGroup|NumberofAlarms|UnitType|Unit

In [ ]:
spark.sql("SELECT distinct upper(CallType) FROM firecalls where \
CallDate = '04/19/2020'and IncidentNumber >20046358 ").show(1000,False)

+-------------------------------+
|upper(CallType)                |
+-------------------------------+
|ALARMS                         |
|OUTSIDE FIRE                   |
|TRAFFIC COLLISION              |
|GAS LEAK (NATURAL AND LP GASES)|
|MEDICAL INCIDENT               |
|OTHER                          |
|STRUCTURE FIRE                 |
|CITIZEN ASSIST / SERVICE CALL  |
|SMOKE INVESTIGATION (OUTSIDE)  |
+-------------------------------+



In [ ]:
spark.sql("SELECT NeighborhoodDistrict, count(NeighborhoodDistrict) AS Neighborhood_Count \
FROM firecalls \
WHERE year(to_date(CallDate, 'MM/dd/yyyy')) = 2020 \
GROUP BY NeighborhoodDistrict \
ORDER BY Neighborhood_Count DESC ").show(5)

+--------------------+------------------+
|NeighborhoodDistrict|Neighborhood_Count|
+--------------------+------------------+
|          Tenderloin|                93|
|     South of Market|                69|
|             Mission|                63|
|      Outer Richmond|                49|
|Financial Distric...|                33|
+--------------------+------------------+
only showing top 5 rows



In [ ]:
from pyspark.sql.functions import *

# Note that PySpark uses the Java Simple Date Format patterns

from_pattern1 = 'MM/dd/yyyy'
to_pattern1 = 'yyyy-MM-dd'

from_pattern2 = 'MM/dd/yyyy hh:mm:ss aa'
to_pattern2 = 'MM/dd/yyyy hh:mm:ss aa'


fireServiceCallsTsDF = fireServiceCallsDF \
  .withColumn('CallDateTS', unix_timestamp(fireServiceCallsDF['Call Date'], from_pattern1).cast("timestamp")) \
  .drop('CallDate') \
  .withColumn('WatchDateTS', unix_timestamp(fireServiceCallsDF['Watch Date'], from_pattern1).cast("timestamp")) \
  .drop('WatchDate') \
  .withColumn('ReceivedDtTmTS', unix_timestamp(fireServiceCallsDF['Received DtTm'], from_pattern2).cast("timestamp")) \
  .drop('ReceivedDtTm')


fireServiceCallsTsDF.printSchema()
fireServiceCallsTsDF.show(5)

root
 |-- Call Number: integer (nullable = true)
 |-- Unit ID: string (nullable = true)
 |-- Incident Number: integer (nullable = true)
 |-- Call Type: string (nullable = true)
 |-- Call Date: string (nullable = true)
 |-- Watch Date: string (nullable = true)
 |-- Received DtTm: string (nullable = true)
 |-- Entry DtTm: string (nullable = true)
 |-- Dispatch DtTm: string (nullable = true)
 |-- Response DtTm: string (nullable = true)
 |-- On Scene DtTm: string (nullable = true)
 |-- Transport DtTm: string (nullable = true)
 |-- Hospital DtTm: string (nullable = true)
 |-- Call Final Disposition: string (nullable = true)
 |-- Available DtTm: string (nullable = true)
 |-- Address: string (nullable = true)
 |-- City: string (nullable = true)
 |-- Zipcode of Incident: integer (nullable = true)
 |-- Battalion: string (nullable = true)
 |-- Station Area: integer (nullable = true)
 |-- Box: integer (nullable = true)
 |-- Original Priority: string (nullable = true)
 |-- Priority: string (nullab